In [ ]:
import requests
import json
import datetime
import pandas as pd

# Twitter API
bearer_token = "Your Bearer Token"

# Decide the Range of Date
end_date = datetime.datetime.now().date()
start_date = end_date - datetime.timedelta(days=6)

tweet_list = []

# Extracting Tweet For last 7 days
for i in range(7):
    current_date = start_date + datetime.timedelta(days=i)

    # to make request for every hour in a day
    for hour in range(23):
        start_time = datetime.datetime.combine(current_date, datetime.time(hour, 0)).isoformat() + "+00:00"
        end_time = datetime.datetime.combine(current_date, datetime.time(hour + 1, 0)).isoformat() + "+00:00"

        # Deciding parameters
        url = "https://api.twitter.com/2/tweets/search/recent" 
        headers = {
            "Authorization": "Bearer " + bearer_token,
            "Content-Type": "application/json"
        }
        params = {
            "query": "#yourquery -is:retweet lang:en",
            "max_results": 100,
            "start_time": start_time,
            "end_time": end_time,
            "tweet.fields": "created_at,author_id,public_metrics,entities"
        }

        # API request
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        
        for tweet in data.get("data", []):
            tweet_id = tweet["id"]
            created_at = tweet["created_at"]
            author_id = tweet["author_id"]
            retweet_count = tweet["public_metrics"]["retweet_count"]
            like_count = tweet["public_metrics"]["like_count"]
            text = tweet["text"]

            
            hashtags = []
            if "entities" in tweet and "hashtags" in tweet["entities"]:
                for hashtag in tweet["entities"]["hashtags"]:
                    hashtags.append(hashtag["tag"])

            tweet_dict = {
                "Tweet ID": tweet_id,
                "Created At": created_at,
                "Author ID": author_id,
                "Retweet Count": retweet_count,
                "Like Count": like_count,
                "Text": text,
                "Hashtags": hashtags
            }
            tweet_list.append(tweet_dict)

        # Controlling rate limits
        rate_limit_remaining = int(response.headers.get("x-rate-limit-remaining"))
        if rate_limit_remaining == 0:
            rate_limit_reset = int(response.headers.get("x-rate-limit-reset"))
            current_time = int(datetime.datetime.now().timestamp())
            sleep_time = rate_limit_reset - current_time
            if sleep_time > 0:
                print(f"Rate limit reached. Waiting for {sleep_time} seconds...")
                time.sleep(sleep_time)

df = pd.DataFrame(tweet_list)
df.to_csv('tweetslist.csv', index=False)
